In [1]:
# Final project final code

In [1]:
!pip install openunmix

  Using cached openunmix-1.2.1-py3-none-any.whl (46 kB)


In [2]:
!pip install soundfile

In [3]:
!pip install tensorflow

  Using cached tensorflow-2.8.0-cp39-cp39-manylinux2010_x86_64.whl (497.6 MB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached libclang-13.0.0-py2.py3-none-manylinux1_x86_64.whl (14.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorflow_io_gcs_filesystem-0.24.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached grpcio-1.44.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Using cached wrapt-1.14.0-cp3

In [4]:
!pip install tensorflow_io

  Using cached tensorflow_io-0.24.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.4 MB)


In [5]:
# Import dependencies

import torch
import torchaudio
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np
import scipy
import os
from IPython.display import Audio, display

import librosa
import sklearn
import math
import torch.nn as nn
import torch.nn.functional as nnF
from tensorflow.keras import layers, models

import soundfile
import librosa.display
import matplotlib.pyplot as plt
from librosa.feature import melspectrogram as mel_spec

from openunmix import predict

# set to GPU if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
# TRAIN/BUILD CNN

# path to training data 
train_ds  = os.path.expanduser("~/teaching_material/own_work/train_data")

# path to drums training data
drum_ds  = os.path.expanduser("~/drum_data/")

In [7]:
# function to shuffle two arrays
def shuffled_arrays(x, y):
    p = np.random.permutation(x.shape[0])
    return x[p], y[p]

In [8]:
# function to convert training data into spectrograms
def convert_audio_to_spectrogram(infile, win_size = 0.05, hop_size = 0.025, n_mels = 64):
   
    # load audio file and get sample rate
    snd, fs = librosa.load(infile)
    
    # compute mel spec parameters
    win_len = int(win_size * fs)
    hop_len = int(hop_size * fs)
    
    # compute mel spectrogram from raw audio
    spec = mel_spec(snd, sr=fs, n_mels=n_mels, hop_length=hop_len, 
                                          win_length=win_len, window='hann')
    
    # rescale output range to be logarithmic
    # corresponds to human perception
    spec = tfio.audio.dbscale(spec, top_db=80)
    
    # augment spectrogram in terms of frequency
    augmented_spec = tfio.audio.freq_mask(spec, param=10)
    
    # convert to original spectrogram to tensor
    # add new axis to account for channels
    spec = spec[..., tf.newaxis]
    spec = tf.convert_to_tensor(spec)
    # reshape to tensor shape
    spec = tf.reshape(spec, (64, 121, 1))
    
    # convert to augmented spectrogram to tensor
    augmented_spec = augmented_spec[..., tf.newaxis]
    augmented_spec = tf.convert_to_tensor(augmented_spec)
    augmented_spec = tf.reshape(augmented_spec, (64, 121, 1))
    
    # output both
    return spec, augmented_spec

In [9]:
# function to get training data labels from file name
def get_label(infile):
    
    # get section of file name associated with instrument
    file_name = infile[1:4] 

    if file_name == 'cel':
        label = [1,0,0,0,0,0,0,0,0,0,0,0,0]
        
    elif file_name == 'cla':
        label = [0,1,0,0,0,0,0,0,0,0,0,0,0]
        
    elif file_name == 'flu':
        label = [0,0,1,0,0,0,0,0,0,0,0,0,0]
        
    elif file_name == 'gac':
        label = [0,0,0,1,0,0,0,0,0,0,0,0,0]
        
    elif file_name == 'gel':
        label = [0,0,0,0,1,0,0,0,0,0,0,0,0]
        
    elif file_name == 'org':
        label = [0,0,0,0,0,1,0,0,0,0,0,0,0]
        
    elif file_name == 'pia':
        label = [0,0,0,0,0,0,1,0,0,0,0,0,0]
        
    elif file_name == 'sax':
        label = [0,0,0,0,0,0,0,1,0,0,0,0,0]
        
    elif file_name == 'tru':
        label = [0,0,0,0,0,0,0,0,1,0,0,0,0]
        
    elif file_name == 'vio':
        label = [0,0,0,0,0,0,0,0,0,1,0,0,0]
        
    elif file_name == 'voi':
        label = [0,0,0,0,0,0,0,0,0,0,1,0,0]
        
    else:
        label = [0,0,0,0,0,0,0,0,0,0,0,0,1]
    
    return label

In [10]:
# store labels names as strings in an array for output use
label_names = ['Cello', 'Clarinet', 'Flute', 'Acoustic Guitar', 'Electric Guitar', 
               'Organ', 'Piano', 'Sax', 'Trumpet', 'Violin', 'Voice', 'Drums', 'Other']

In [11]:
# function to convert vectorised labels back to words
def vector_to_label(vector):
    
    word = "Undefined"
    
    if vector[0] == 1:
        word = "Cello"
        
    elif vector[1] == 1:
        word = "Clarinet"
        
    elif vector[2] == 1:
        word = "Flute"
            
    elif vector[3] == 1:
        word = "Acoustic Guitar"
                
    elif vector[4] == 1:
        word = "Electric Guitar"
                    
    elif vector[5] == 1:
        word = "Organ"
                        
    elif vector[6] == 1:
        word = "Piano"
                            
    elif vector[7] == 1:
        word = "Sax"
                                
    elif vector[8] == 1:
        word = "Trumpet"
                                    
    elif vector[9] == 1:
        word = "Violin"
                                        
    elif vector[10] == 1:
        word = "Voice"
        
    elif vector[11] == 1:
        word = "Drums"

    return word

In [12]:
# check total number of training samples
no_of_entries = len([name for name in os.listdir(train_ds)])
print(no_of_entries)

3261


In [13]:
# initialise an array for storing output spectrogram tensors
spectrograms = np.zeros((no_of_entries * 2, 64, 121, 1))

# initialise counter for counting number of spectrograms calculated
# also used for location for storing spectrograms
counter = 0

# scan through training data directory and compute spectrograms
for entry in os.scandir(train_ds):
    
    # if file is not .wav then skip
    if str(os.listdir(train_ds)[counter][-3:]) != 'wav':
        continue
    
    # add output spectrograms to spectrogram array
    spec, augmented_spec = convert_audio_to_spectrogram(entry)
    spectrograms[counter] = spec
    spectrograms[counter + 1] = augmented_spec
    # advance counter
    counter += 2
    
    # print progress
    if counter % 50 == 0:
        print(counter)

2022-04-12 14:04:51.204164: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-12 14:05:10.629022: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 41734 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:17:00.0, compute capability: 8.6
2022-04-12 14:05:10.630817: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 41734 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:65:00.0, compute capability: 8.6
2022-04-12 14:05:10.632516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750


In [14]:
# initalise array for storing label data
labels = np.zeros((no_of_entries * 2, 13))

# counter for inputting labels in array
label_counter = 0

for name in os.listdir(train_ds):
    
    # skip file if not .wav
    if str(os.listdir(train_ds)[label_counter][-3:]) != 'wav':
        continue
    
    # output labels to array
    labels[label_counter] = get_label(name)
    labels[label_counter + 1] = get_label(name)    
    # advance counter
    label_counter += 2

In [15]:
# get spectrograms and label arrays to correct length
spectrograms = spectrograms[:counter]
labels = labels[:label_counter]

# get total number of samples from spectrogram counter
total_length = counter

# print(counter)

In [16]:
drum_spectrograms = []

for drum_file in os.scandir(drum_ds):
    
    spec_counter = 0
    
    snd, sr = librosa.load(drum_file)
    
    three_seconds = 3 * sr
    n_mels = 64
    win_len = int(0.05 * sr)
    hop_len = int(0.025 * sr)
    
    for i in range(len(snd)):
        
        if spec_counter > 13:
            break
        
        if (i % three_seconds) == 0:

            # compute mel spectrogram from raw audio
            spec = mel_spec(snd[i:i+three_seconds], sr=sr, n_mels=n_mels, hop_length=hop_len, 
                                                  win_length=win_len, window='hann')
            
            spec_counter += 1

            if(spec.shape[1] < 121):
                continue
            
            # rescale output range to be logarithmic
            # corresponds to human perception
            spec = tfio.audio.dbscale(spec, top_db=80)

            # augment spectrogram in terms of frequency
            augmented_spec = tfio.audio.freq_mask(spec, param=10)

            # convert to original spectrogram to tensor
            # add new axis to account for channels
            spec = spec[..., tf.newaxis]
            spec = tf.convert_to_tensor(spec)
            # reshape to tensor shape
            spec = tf.reshape(spec, (64, 121, 1))
            
            # convert to augmented spectrogram to tensor
            augmented_spec = augmented_spec[..., tf.newaxis]
            augmented_spec = tf.convert_to_tensor(augmented_spec)
            augmented_spec = tf.reshape(augmented_spec, (64, 121, 1))
           
            drum_spectrograms.append(spec)
            drum_spectrograms.append(augmented_spec)
            
            
print(len(drum_spectrograms))

# drum_spectrograms = drum_spectrograms[:300]

# print(len(drum_spectrograms))

/opt/conda/lib/python3.9/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=184
  warnings.warn(


520


In [17]:
drum_spectrograms = drum_spectrograms[:250]

In [18]:
# print(drum_spectrograms.shape)

In [19]:
drum_labels = np.zeros((len(drum_spectrograms), 13))

for i in range(len(drum_spectrograms)):
    drum_labels[i] = [0,0,0,0,0,0,0,0,0,0,0,1,0]
    
print(drum_labels[-1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [20]:
# get largest value from across spectrograms
most_max_regular = np.amax(spectrograms)

print(most_max_regular)

# normalise all specs according to maximum value
# maybe i will take away the floor dividing
normalised_specs = np.abs(spectrograms // most_max_regular)
normalised_specs = np.where(np.isnan(normalised_specs), 0, normalised_specs)

# same for drums
most_max_drums = np.amax(drum_spectrograms)

print(most_max_drums)

# if np.any(np.isnan(drum_spectrograms)):
#     print("bruh")

# normalise all specs according to maximum value
normalised_drum_specs = np.abs(drum_spectrograms // most_max_drums)
normalised_drum_specs = np.where(np.isnan(normalised_drum_specs), 0, normalised_drum_specs)

66.93445587158203
67.121185


In [21]:
# TO DO :

all_spectrograms = [normalised_specs, normalised_drum_specs]
all_spectrograms = np.concatenate(all_spectrograms)
all_labels = [labels, drum_labels]
all_labels = np.concatenate(all_labels)

print(all_spectrograms.shape)
print(all_labels.shape)

(3044, 64, 121, 1)
(3044, 13)


In [22]:
# shuffle spectrograms and labels with same permutation 
suffled_spectrograms, shuffled_labels = shuffled_arrays(all_spectrograms, all_labels)

In [23]:
# split data into train and test
# make train data 80% of all data
train_specs = suffled_spectrograms[:int(total_length * 0.8)]

# val data is 10%
val_specs = suffled_spectrograms[int(total_length * 0.8):int(total_length * 0.9)]

# test data is the last 10%
test_specs = suffled_spectrograms[int(total_length * 0.9):total_length]

# split labels same way as spectrograms
train_labels = shuffled_labels[:int(total_length * 0.8)]
val_labels = shuffled_labels[int(total_length * 0.8):int(total_length * 0.9)]
test_labels = shuffled_labels[int(total_length * 0.9):total_length]

# convert spectrograms into tensor for training
train_dataset = tf.data.Dataset.from_tensor_slices(train_specs)

In [24]:
# BUILD CNN MODEL

# use He initialisation
initializer = tf.keras.initializers.HeNormal()

# set up model as sequential keras model
model = models.Sequential()

# add convolutional layer followed by max pooling
model.add(layers.Conv2D(32, (2, 2), activation='tanh', input_shape=(64, 121, 1), kernel_initializer=initializer))
model.add(layers.MaxPooling2D((2, 2)))

# add convolutional layer followed by max pooling
model.add(layers.Conv2D(64, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))

# add convolutional layer followed by max pooling
model.add(layers.Conv2D(128, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))

# normalise the batch 
model.add(layers.BatchNormalization(momentum = 0.9))

# add convolutional layer - this is ususally 16 not 128
model.add(layers.Conv2D(128, (2, 2), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

# flatten
model.add(layers.Flatten())

# three dense layers in sequence to 12 possibel outputs
# model.add(layers.Dense(128, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
# model.add(layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.Dense(512, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.Dense(13, activation='softmax'))

# summarise models
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 63, 120, 32)       160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 60, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 59, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 29, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 28, 128)       32896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 7, 14, 128)       0

In [25]:
# define function to perform step down learning rate decay

# code taken from stack overflow: REFERENCE

def step_decay(epoch):
    initial_lrate = 0.00001
    drop = 0.5
    epochs_drop = 50.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [26]:
# TRAIN THE NETWORK

# initialise optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001, momentum = 0.1)
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile the model
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
)

# train the model using step down learning rate decay
callbacks = [
             tf.keras.callbacks.ModelCheckpoint(
                  filepath='./training_checkpoints/ckpt_{epoch}',
                  save_weights_only=True
             ),
            tf.keras.callbacks.LearningRateScheduler(
                step_decay, 
                verbose=1
            ),
]

# train the model
history = model.fit(train_specs, train_labels, epochs=100, callbacks=callbacks,
                   validation_data=(val_specs, val_labels))


Epoch 1: LearningRateScheduler setting learning rate to 1e-05.
Epoch 1/100


2022-04-12 14:15:45.692939: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2022-04-12 14:15:47.480993: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-12 14:15:47.482626: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-12 14:15:47.482694: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-04-12 14:15:47.484377: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-12 14:15:47.484535: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-04-12 14:15:48.791892: I tensorflow/stream_executor/cuda/c

70/70 [==============================] - 7s 21ms/step - loss: 5.0812 - accuracy: 0.2116 - val_loss: 4.8575 - val_accuracy: 0.3154 - lr: 1.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 1e-05.
Epoch 2/100
70/70 [==============================] - 1s 17ms/step - loss: 4.6920 - accuracy: 0.3826 - val_loss: 4.6520 - val_accuracy: 0.4050 - lr: 1.0000e-05

Epoch 3: LearningRateScheduler setting learning rate to 1e-05.
Epoch 3/100
70/70 [==============================] - 1s 17ms/step - loss: 4.4543 - accuracy: 0.5025 - val_loss: 4.5745 - val_accuracy: 0.4444 - lr: 1.0000e-05

Epoch 4: LearningRateScheduler setting learning rate to 1e-05.
Epoch 4/100
70/70 [==============================] - 1s 18ms/step - loss: 4.2522 - accuracy: 0.5928 - val_loss: 4.4410 - val_accuracy: 0.4624 - lr: 1.0000e-05

Epoch 5: LearningRateScheduler setting learning rate to 1e-05.
Epoch 5/100
70/70 [==============================] - 1s 18ms/step - loss: 4.0777 - accuracy: 0.6617 - val_loss: 4.3777 -

2022-04-12 14:17:04.449800: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at save_restore_v2_ops.cc:160 : RESOURCE_EXHAUSTED: training_checkpoints/ckpt_52_temp/part-00000-of-00001.data-00000-of-00001.tempstate9833815605142357114; No space left on device


ResourceExhaustedError: training_checkpoints/ckpt_52_temp/part-00000-of-00001.data-00000-of-00001.tempstate9833815605142357114; No space left on device [Op:SaveV2]

In [159]:
# EVAULATION PORTION OF THE MODEL
# run test data through model with ground truths
# test_loss, test_acc = model.evaluate(test_specs,  test_labels, verbose=2)

# get predictions of test spectrograms 
predictions = model.predict(test_specs)

In [160]:
# print(predictions[7])
# print()

In [161]:
# function to calculate accuracies of the model
# outputs true positive, false positives, true negatives and false negatives
# compares ground truths to predictions for specified instrument
def counters(ground_truths, predictions, instrument):
    
    # initialise counters for each outcome
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    # cycle through output predictions
    for i in range(len(predictions)):
        
        # check if prediction matches specified instrument
        if predictions[i] == instrument:
            
            # if ground truth matches instrument, add one to true positives
            if ground_truths[i] == instrument:
                TP += 1
                
            # else if ground truth does not match instrument, add one to false positives
            if ground_truths[i] != instrument:
                FP += 1
                print("False Positive for: ", instrument)
                print(ground_truths[i])
            
        # check if prediction does not match specified instrument
        if predictions[i] != instrument:
            
            # if ground truth also does not match instrument, add one to true negatives
            if ground_truths != instrument:
                TN += 1
            
            # else if ground truth is instrument, add one to false negatives
            if ground_truths[i] == instrument:
                FN += 1
                # print("False Negative for: ", instrument)
                # print(predictions[i])
            
    return TP, FP, TN, FN

In [162]:
# get number of predictions made
no_of_predictions = len(predictions)

# intialise arrays for storing labels and classifications as words
true_labels = []
classification = []

# loop through all predictions
for i in range(no_of_predictions):  
    # convert vector labels to word labels
    new_true = vector_to_label(test_labels[i])
    # print(new_true)
    true_labels.append(new_true)
    
    # get index where vector equals one
    # vector should be 1-hot encoded
    label_no = np.argmax(predictions[i])
    # output word classification to array
    # print(label_names[label_no])
    classification.append(label_names[label_no])
    
# check stats for classification vs ground truth for each instrument
for label in label_names:
    TP, FP, TN, FN = counters(true_labels, classification, label)

    print(label, "TP:", TP)
    print(label, "FP:",  FP)
    print(label, "TN:", TN)
    print(label, "FN:", FN)
    print(" ")

False Positive for:  Cello
Acoustic Guitar
False Positive for:  Cello
Electric Guitar
False Positive for:  Cello
Voice
False Positive for:  Cello
Piano
False Positive for:  Cello
Voice
Cello TP: 25
Cello FP: 5
Cello TN: 250
Cello FN: 3
 
False Positive for:  Clarinet
Violin
False Positive for:  Clarinet
Flute
False Positive for:  Clarinet
Electric Guitar
False Positive for:  Clarinet
Flute
False Positive for:  Clarinet
Organ
False Positive for:  Clarinet
Organ
False Positive for:  Clarinet
Sax
False Positive for:  Clarinet
Flute
False Positive for:  Clarinet
Violin
False Positive for:  Clarinet
Violin
Clarinet TP: 22
Clarinet FP: 10
Clarinet TN: 248
Clarinet FN: 5
 
False Positive for:  Flute
Acoustic Guitar
False Positive for:  Flute
Acoustic Guitar
Flute TP: 15
Flute FP: 2
Flute TN: 263
Flute FN: 8
 
False Positive for:  Acoustic Guitar
Clarinet
False Positive for:  Acoustic Guitar
Sax
False Positive for:  Acoustic Guitar
Electric Guitar
False Positive for:  Acoustic Guitar
Flute
Aco

In [163]:
# OPEN-UNMIX PORTION OF THE CODE

In [139]:
# TO DO:

# IMPORT CASE STUDY FILE AND SEND THROUGH OPEN-UNMIX

In [140]:
# TO DO:

# add potion to upload input file

# test_audio = "bensound-clapandyell.mp3"

# case_study_snd, sr = librosa.load(test_audio)

In [141]:
# initialise array for storing output audio
# audio = []

# # having some issues with that there sample rate
# # sr = 22050

# estimates = predict.separate(
#     torch.as_tensor(case_study_snd).float(),
#     rate = sr,
#     device = device
# )
# for target, estimates in estimates.items():
#     print(target)
#     output = estimates.detach().cpu().numpy()[0]
#     audio.append(output)
#     display(Audio(output, rate=sr*2))
#     # had to transpose output for it to not be empty
#     # file_name = target + '.wav'
#     # soundfile.write(file_name, audio.T, 16000)

In [37]:
# TO DO:

# SDR etc evaluations on Open-Unmix output
# museval toolkit

In [38]:
# TO DO:

# PRE-PROCESS EACH OUTPUT TO BE CORRECT LENGTH
# CONVERT SECTIONS TO SPECTROGRAMS
# SEND ALL CUTS THROUGH CNN
# TOTAL UP CLASSIFICATIONS FOR CUTS THAT ARE ALL SAME SOURCE
# OUTPUT CLASSIFICATION AS MOST COMMON FOR THAT SOURCE


In [39]:
audio = []

filenames = ["vocals.wav","drums.wav","bass.wav","other.wav"]

for file in filenames:    
    snd, sr = librosa.load(file)
    audio.append(snd)


In [40]:
print(len(audio))

4


In [42]:
three_seconds = 3 * sr

case_study_spectrograms = []
# compute mel spec parameters
win_len = int(0.05 * sr)
hop_len = int(0.025 * sr)
n_mels = 64

for n, output in enumerate(audio):
    spec_counter = 0
    
    # output_mono = librosa.to_mono(output)
    
    for i in range(len(snd)):
        if (i % three_seconds) == 0:

            # compute mel spectrogram from raw audio
            spec = mel_spec(output[i:i+three_seconds], sr=sr, n_mels=n_mels, hop_length=hop_len, 
                                                  win_length=win_len, window='hann')

            if(spec.shape[1] < 121):
                continue
            
            # rescale output range to be logarithmic
            # corresponds to human perception
            spec = tfio.audio.dbscale(spec, top_db=80)

            # augment spectrogram in terms of frequency
            augmented_spec = tfio.audio.freq_mask(spec, param=10)

            # convert to original spectrogram to tensor
            # add new axis to account for channels
            spec = spec[..., tf.newaxis]
            spec = tf.convert_to_tensor(spec)
            # reshape to tensor shape
            spec = tf.reshape(spec, (64, 121, 1))
           
            case_study_spectrograms.append(spec)
            spec_counter += 1
        
    print(n, ":", spec_counter)
    
print("Total no of spectrograms: ", len(case_study_spectrograms))

0 : 58
1 : 58
2 : 58
3 : 58
Total no of spectrograms:  232


In [65]:
# get largest value from across spectrograms
case_study_max = np.amax(case_study_spectrograms)

print(case_study_max)

# normalise all specs according to maximum value
normalised_case_study = np.abs(case_study_spectrograms // case_study_max)
normalised_case_study = np.where(np.isnan(normalised_case_study), 0, normalised_case_study)

63.44747


In [142]:
case_study_predictions = model.predict(normalised_case_study)

case_study_classifications = []

for k in range(len(case_study_predictions)):
    case_label_no = np.argmax(predictions[k])
    # output word classification to array
    case_study_classifications.append(label_names[case_label_no])

In [143]:
voice_class = case_study_classifications[:58]
drums_class = case_study_classifications[58:116]
bass_class = case_study_classifications[116:174]
other_class = case_study_classifications[174:]

unique_voice = np.unique(voice_class, return_counts=True)
print(unique_voice)
max_value_location = np.argmax(unique_voice[1])
final_voice_classification = unique_voice[0][max_value_location]
print(final_voice_classification)

unique_drums = np.unique(drums_class, return_counts=True)
print(unique_drums)
max_value_location_d = np.argmax(unique_drums[1])
final_drums_classification = unique_drums[0][max_value_location_d]
print(final_drums_classification)

unique_bass = np.unique(bass_class, return_counts=True)
print(unique_bass)
max_value_location_b = np.argmax(unique_bass[1])
final_bass_classification = unique_bass[0][max_value_location_b]
print(final_bass_classification)

unique_other = np.unique(other_class, return_counts=True)
print(unique_other)
max_value_location_o = np.argmax(unique_other[1])
final_other_classification = unique_other[0][max_value_location_o]
print(final_other_classification)

(array(['Acoustic Guitar', 'Cello', 'Clarinet', 'Drums', 'Electric Guitar',
       'Flute', 'Organ', 'Piano', 'Sax', 'Trumpet', 'Violin', 'Voice'],
      dtype='<U15'), array([ 6,  2,  5, 12,  5,  2,  3,  3,  8,  6,  3,  3]))
Drums
(array(['Acoustic Guitar', 'Cello', 'Clarinet', 'Drums', 'Electric Guitar',
       'Flute', 'Organ', 'Piano', 'Sax', 'Trumpet', 'Violin', 'Voice'],
      dtype='<U15'), array([ 3,  2,  4, 11,  4,  3,  7,  5,  5,  3,  5,  6]))
Drums
(array(['Cello', 'Clarinet', 'Drums', 'Electric Guitar', 'Flute', 'Organ',
       'Piano', 'Sax', 'Trumpet', 'Violin', 'Voice'], dtype='<U15'), array([4, 5, 8, 7, 3, 6, 3, 8, 9, 3, 2]))
Trumpet
(array(['Acoustic Guitar', 'Cello', 'Clarinet', 'Drums', 'Electric Guitar',
       'Flute', 'Organ', 'Piano', 'Sax', 'Trumpet', 'Violin', 'Voice'],
      dtype='<U15'), array([4, 8, 5, 7, 5, 6, 4, 5, 5, 2, 3, 4]))
Cello
